# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file from part 1
# Not sure if I should be using the cleaned one or the previous one, 
#but Im guessing the cleaned one
file = "../Output_Data/Clean_City_Data"
clean_df = pd.read_csv(file)
# clean_df has a 'unnamed' column, so remove it so it looks better
#clean_df.head()
weather_df = clean_df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.80,-68.30,53.60,46,75,12.75,AR,1603317621
1,katsuura,35.13,140.30,60.80,67,75,13.87,JP,1603317044
2,mongo,12.18,18.69,81.59,30,69,4.70,TD,1603317621
3,cap malheureux,-19.98,57.61,71.01,86,74,5.99,MU,1603317405
4,chokurdakh,70.63,147.92,9.93,94,100,4.36,RU,1603317035


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Initiate variables
locations = weather_df[['Lat', 'Lng']].astype(float)
humidity = weather_df['Humidity'].astype(float)

In [4]:
# Add some specifications to heatmap
heatmap_specs = {
    'width': '1000px',
    'height': '500px',
    'margin': '0 auto 0 auto'
}

# Create map
fig = gmaps.figure(layout=heatmap_specs, zoom_level=2, center=(0,0))

# Add layer details
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100, 
                                 point_radius=1)
fig.add_layer(heat_layer)

plt.savefig("../Images/humidty_heatmap.png")

fig

Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Set specifications
ideal_temp = (weather_df['Max Temp']>70) & (weather_df['Max Temp']<86)
ideal_wind = weather_df['Wind Speed']<10
ideal_humid = weather_df['Humidity']<50

# Collect all ideal specs
ideal_vaca = ideal_temp & ideal_wind & ideal_humid

# Create new df using collected specs
ideal_vaca_df = weather_df[ideal_vaca]
ideal_vaca_df = ideal_vaca_df.dropna()

# Limit the number of rows returned by your API requests to a reasonable number.
# I think this is what that means... by making sure only 7 rows are returned..
ideal_vaca_df.head(7)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,mongo,12.18,18.69,81.59,30,69,4.70,TD,1603317621
22,fort abbas,29.19,72.85,72.30,21,0,7.90,PK,1603317626
98,susanville,40.42,-120.65,78.01,19,1,3.36,US,1603317728
161,am timan,11.03,20.28,75.69,45,98,3.71,TD,1603317845
249,bandar-e lengeh,26.56,54.88,80.60,28,0,9.17,IR,1603317926
254,kassala,15.45,36.40,81.43,42,0,8.39,SD,1603317927
273,aswan,24.09,32.91,74.98,31,0,1.97,EG,1603317932


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
# I think that means just change the name of the df..
hotel_df = ideal_vaca_df

# Add a "Hotel Name" column to the df
# Use empty quotes for initial value, since we don't have that data yet
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,mongo,12.18,18.69,81.59,30,69,4.70,TD,1603317621,
22,fort abbas,29.19,72.85,72.30,21,0,7.90,PK,1603317626,
98,susanville,40.42,-120.65,78.01,19,1,3.36,US,1603317728,
161,am timan,11.03,20.28,75.69,45,98,3.71,TD,1603317845,
249,bandar-e lengeh,26.56,54.88,80.60,28,0,9.17,IR,1603317926,


In [7]:
# Hit the Google Places API for each city's coordinates
# Set parameters dictionary to search for hotels with 5000 meters
params = {
    "radius": 5000, 
    "types": "hotels", 
    "keyword": "hotel", 
    "key": g_key}

# Start a for loop using iterrows 
for index, row in hotel_df.iterrows():
    
    # First, get the lat and long coords from our df
    lat = row['Lat']
    lng = row['Lng']
    
    # Add a location parameter using lat and long that we just iterrated through
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    
    # Store the first Hotel result into the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except:
        print("Missing data")

Missing data
Missing data
Missing data
Missing data


In [8]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,mongo,12.18,18.69,81.59,30,69,4.70,TD,1603317621,
22,fort abbas,29.19,72.85,72.30,21,0,7.90,PK,1603317626,Shangrila Hotel & Guest House
98,susanville,40.42,-120.65,78.01,19,1,3.36,US,1603317728,Red Lion Inn & Suites Susanville
161,am timan,11.03,20.28,75.69,45,98,3.71,TD,1603317845,
249,bandar-e lengeh,26.56,54.88,80.60,28,0,9.17,IR,1603317926,Diplomat Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add some specifications to heatmap
# This part reminds me of CSS, syntax is basicly identical, which makes sense since we are styling an image
heatmap_specs = {
    'width': '1000px',
    'height': '500px',
    'margin': '0 auto 0 auto'
}

# Add marker layer ontop of heat map
fig = gmaps.figure(layout=heatmap_specs, zoom_level=2, center=(0,0))

hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# Save figure
plt.savefig("../Images/hotel_heatmap.png")

# Display figure
fig

Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>